# Pose Corrector for Squats

## Importing Libraries

In [2]:
from calendar import c
import time
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
import asyncio

## Functions for angle calculations and rescaling frames

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent / 100)
    height = int(frame.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

### Pose Corrector

In [4]:


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose




angle_min = []  # list of all the min angles of the knee joint
# min angle of the knee joint is the angle when you are standing straight and max
angle_min_hip = []
angle_min_ankle = []
angle_min_trunk = []

cap = cv2.VideoCapture('demos/Squat/squat.mp4')

# Curl counter variables
counter = 0
min_ang = 0
max_ang = 0
min_ang_hip = 0
max_ang_hip = 0
stage = None

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (540, 380)

min_angle_min_ankle = 0  # Initialize min_angle_min_ankle
max_angle_min_ankle = 0  # Initialize max_angle_min_ankle
min_angle_min_trunk = 0  # Initialize min_angle_min_trunk
max_angle_min_trunk = 0  # Initialize max_angle_min_trunk


# msg_id_displayed = False
with mp_pose.Pose(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if frame is not None:
            frame_ = rescale_frame(frame, percent=75)

        # Recolor image to RGB
        image = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # get coordinates
            index = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            # get coordinates
            index_r = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            angle_knee = calculate_angle(hip, knee, ankle)  # Knee joint angle
            angle_knee = round(angle_knee, 2)
            print("Knee angle", angle_knee)

            angle_hip = calculate_angle(shoulder, hip, knee)
            angle_hip = round(angle_hip, 2)

            angle_ankle = calculate_angle(knee, ankle, index_r)  # Ankle joint angle
            angle_ankle = round(angle_ankle, 2)
            print("Ankle angle", angle_ankle)

            angle_trunk = calculate_angle(hip, shoulder, elbow)
            angle_trunk = round(angle_trunk, 2)

            hip_angle = 180 - angle_hip  # Hip angle
            knee_angle = 180 - angle_knee
            ankle_angle = 180 - angle_ankle
            trunk_angle = 180 - angle_trunk

            angle_min.append(angle_knee)  # append all the angles of the knee joint
            angle_min_hip.append(angle_hip)
            angle_min_ankle.append(angle_ankle)
            angle_min_trunk.append(angle_trunk)

            print(angle_knee)
            print(angle_hip)
            print(angle_ankle)
            print(angle_trunk)

            # Visualize angle
            # changing the color of text from white to black

            cv2.putText(image, str(angle), 

                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(angle_knee),
                        tuple(np.multiply(knee, [1500, 800]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA
                        )

            cv2.putText(image, str(angle_hip),
                        tuple(np.multiply(hip, [1500, 800]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA
                        )

            cv2.putText(image, str(angle_ankle),
                        tuple(np.multiply(ankle, [1500, 750]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA
                        )

            cv2.putText(image, str(angle_trunk),
                        tuple(np.multiply(shoulder, [1500, 400]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA
                        )

            # Curl counter logic
            if angle_knee > 90:
                stage = "up"
            if angle_knee <= 90 and stage == 'up':  # 90 is the angle when you are standing straight   and 180 is the angle when you are sitting down
                stage = "down"
                counter += 1
                print(counter)
                min_ang = min(angle_min)  # min angle of the knee joint
                max_ang = max(angle_min)  # max angle of the knee joint

                min_ang_hip = min(angle_min_hip)  # min angle of hip joint
                max_ang_hip = max(angle_min_hip)

                min_angle_min_ankle = min(angle_min_ankle)  # min angle of ankle joint
                max_angle_min_ankle = max(angle_min_ankle)

                min_angle_min_trunk = min(angle_min_trunk)  # min angle of trunk joint
                max_angle_min_trunk = max(angle_min_trunk)

                print(min(angle_min), " _ ", max(angle_min))
                print(min(angle_min_hip), " _ ", max(angle_min_hip))
                print(min(angle_min_ankle), " _ ", max(angle_min_ankle))
                print(min(angle_min_trunk), " _ ", max(angle_min_trunk))
                org = (20, 20)
                org1 = (20, 35)
                org2 = (20, 50)
                org3 = (20, 65)
                org4 = (20, 75)
                org5 = (20, 90)
                org6 = (20, 105)
                org7 = (20, 115)
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontscale = 1
                color = (123, 125, 15)
                thickness = 2
                lineType = cv2.LINE_AA

                # Check angles for correction
            if stage == "down":      
                if hip_angle < 50:
                    cv2.putText(image, 'Hip angle is too narrow', org, font, fontscale, color, thickness, lineType)

                if angle_hip > 150:
                    cv2.putText(image, 'Hip angle is too wide', org4, font, fontscale, color, thickness, lineType)

                if knee_angle < 60:
                    cv2.putText(image, 'Knee angle is too narrow', org2, font, fontscale, color, thickness, lineType)

                if knee_angle > 75:
                    cv2.putText(image, 'Knee angle is too wide', org5, font, fontscale, color, thickness, lineType)

                msg_id = 12345  # Define the variable "msg_id"
                if angle_ankle < 100:
                    cv2.putText(image, 'ankle angle is too narrow', org6, font, fontscale, color, thickness, lineType)

        except:
             pass
        # Render squat counter
        # Setup status box

        # Rep data
        rep_text = "Repetition : " + str(counter)
        cv2.putText(image, 'REPS', (1000, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 128, 128), 1, cv2.LINE_AA)
        cv2.putText(image, rep_text,
                    (1000, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE : ', (850, 300),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (1000, 300),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Knee angle:
        cv2.putText(image, 'Angle', (1000, 10),  # what is 65,12 here ? it is the position of the text on the screen change it to top right corner by changing the values which are 65,12 and
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, "Knee-joint angle : " + str(min_ang),
                    (1000, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Hip angle:
        cv2.putText(image, "Hip-joint angle : " + str(min_ang_hip),
                    (1000, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2, cv2.LINE_AA)
        # Ankle angle:
        cv2.putText(image, "Ankle-joint angle : " + str(min_angle_min_ankle),
                    (1000, 180),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Trunk angle:
        cv2.putText(image, "Trunk-joint angle : " + str(min_angle_min_trunk),
                    (1000, 220),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(203, 17, 17), thickness=2, circle_radius=1)
                                  )

   #     out.write(image)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            break

cap.release()
out.release()
cv2.destroyAllWindows()


Knee angle 176.83
Ankle angle 131.67
176.83
170.28
131.67
15.08
Knee angle 176.56
Ankle angle 131.54
176.56
170.09
131.54
15.64
Knee angle 175.9
Ankle angle 131.39
175.9
169.55
131.39
11.94
Knee angle 174.0
Ankle angle 131.0
174.0
167.07
131.0
11.22
Knee angle 172.44
Ankle angle 130.99
172.44
165.04
130.99
10.52
Knee angle 171.5
Ankle angle 131.72
171.5
163.18
131.72
5.19
Knee angle 171.52
Ankle angle 130.9
171.52
162.11
130.9
6.85
Knee angle 172.79
Ankle angle 130.55
172.79
161.94
130.55
10.26
Knee angle 174.99
Ankle angle 129.52
174.99
161.96
129.52
14.61
Knee angle 177.41
Ankle angle 122.3
177.41
162.28
122.3
14.79
Knee angle 178.38
Ankle angle 114.85
178.38
160.67
114.85
15.89
Knee angle 178.48
Ankle angle 111.91
178.48
160.03
111.91
13.76
Knee angle 178.03
Ankle angle 111.38
178.03
160.3
111.38
14.13
Knee angle 175.23
Ankle angle 110.89
175.23
160.29
110.89
20.54
Knee angle 172.41
Ankle angle 115.1
172.41
159.63
115.1
25.38
Knee angle 170.31
Ankle angle 128.2
170.31
159.17
128.2
2

Knee angle 89.82
Ankle angle 110.92
89.82
98.98
110.92
2.43
1
89.82  _  179.99
98.98  _  175.75
110.89  _  143.5
0.19  _  32.11
Knee angle 84.98
Ankle angle 110.79
84.98
94.41
110.79
1.69
Knee angle 80.9
Ankle angle 110.63
80.9
90.37
110.63
1.8
Knee angle 76.86
Ankle angle 110.6
76.86
86.49
110.6
0.61
Knee angle 72.16
Ankle angle 110.81
72.16
81.37
110.81
0.07
Knee angle 66.88
Ankle angle 110.59
66.88
76.1
110.59
0.28
Knee angle 61.29
Ankle angle 111.34
61.29
70.34
111.34
0.24
Knee angle 58.33
Ankle angle 111.25
58.33
67.08
111.25
1.01
Knee angle 56.49
Ankle angle 110.76
56.49
64.88
110.76
1.91
Knee angle 54.26
Ankle angle 112.29
54.26
62.0
112.29
1.94
Knee angle 54.91
Ankle angle 113.08
54.91
61.98
113.08
2.67
Knee angle 57.41
Ankle angle 112.84
57.41
64.64
112.84
2.42
Knee angle 59.5
Ankle angle 112.71
59.5
66.7
112.71
1.73
Knee angle 61.44
Ankle angle 112.15
61.44
68.79
112.15
1.59
Knee angle 63.95
Ankle angle 112.08
63.95
71.03
112.08
0.57
Knee angle 72.38
Ankle angle 111.25
72.38


NameError: name 'out' is not defined